In [9]:
import metview.bindings
import xarray as xr

ml_wind = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/co/model-level-wind.zarr/",
    chunks={"time": 48},
    consolidated=True,
)

print(f"Model wind dataset size {ml_wind.nbytes / (1024 ** 4):.2f} TiB")
ml_surface

Model wind dataset size 305.89 TiB


<xarray.Dataset>
Dimensions:     (time: 374016, values: 410240)
Coordinates:
    hybrid      float64 ...
    step        timedelta64[ns] ...
  * time        (time) datetime64[ns] 1979-01-01 ... 2021-08-31T23:00:00
    valid_time  (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    lnsp        (time, values) float32 dask.array<chunksize=(48, 410240), meta=np.ndarray>
    z           (time, values) float32 dask.array<chunksize=(48, 410240), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-09-28T00:04 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  62dcc83733d69bb9c25d180532d8911856761df3c788e5...
    pangeo-forge:recipe_hash:  ef1235c98ebe08c72744fe0ec5611b2e307a1087ec8b26...
    pangeo-forge:version:      0.9.1

In [12]:
datestring = "1987-10-16T03"
wind_slice = ml_wind.sel(time=slice(datestring, datestring)).compute()
print(f"Wind slice size {wind_slice.nbytes / (1024 ** 3):.02f} GiB")

'Wind slice size 0.84 GiB'

In [16]:
import metview as mv
# Needs installation of metview binary

def attribute_fix(ds):
    """Needed to fix a low-level bug in ecCodes.
    
    Sometimes, shortNames get overloaded in ecCodes's table. 
    To eliminate ambiguity in their string matching, we
    force ecCodes to make use of the paramId, which is a
    consistent source-of-truth.
    """
    for var in ds:
        attrs = ds[var].attrs
        attrs.pop("GRIB_cfName", None)
        attrs.pop("GRIB_cfVarName", None)
        attrs.pop("GRIB_shortName", None)
        ds[var].attrs.update(attrs)
    return ds

In [18]:
wind_fieldset = mv.dataset_to_fieldset(attribute_fix(wind_slice).squeeze())

/Users/bhavyemathur/DataspellProjects/wind-prediction-merra2/venv/lib/python3.11/site-packages/cfgrib/xarray_to_grib.py:261: FutureWarning: GRIB write support is experimental, DO NOT RELY ON IT!
  warnings.warn("GRIB write support is experimental, DO NOT RELY ON IT!", FutureWarning)


In [19]:
uv_wind_spectral = mv.uvwind(data=wind_fieldset,truncation=639)
uv_wind_gg = mv.read(data=uv_wind_spectral,grid="N320")
uv_wind = mv.read(data=uv_wind_gg, grid=[0.25, 0.25])
uv_wind

In [31]:
uv_wind.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
u,hybrid,"1,2,...",19871016,300,0,None,131,None,None,None,None
v,hybrid,"1,2,...",19871016,300,0,None,132,None,None,None,None


In [28]:
wind_np = uv_wind.values()
wind_np.shape

(274, 1038240)